In [1]:
import pandas as pd
import os
import numpy as np
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from collections import Counter
from sklearn.ensemble import IsolationForest

C:\Users\Miche\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites'
os.chdir(path)
os.getcwd()

'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites'

In [3]:
df_2014 = pd.read_csv('data/yelp_reviews_RV_categories_2014.csv')
df_2015 = pd.read_csv('data/yelp_reviews_RV_categories_2015.csv')
df_2016 = pd.read_csv('data/yelp_reviews_RV_categories_2016.csv')
df_2017 = pd.read_csv('data/yelp_reviews_RV_categories_2017.csv')
df_2018 = pd.read_csv('data/yelp_reviews_RV_categories_2018.csv')
df_2019 = pd.read_csv('Restaurant-Guide/data/yelp_reviews_RV_categories_2019.csv')
df_2020 = pd.read_csv('Restaurant-Guide/data/yelp_reviews_RV_categories_2020.csv')

df = df_2014.append([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020])
# df = df.drop(['hours','address', 'username'],axis=1) #these coulmns are removed, since they are not used. 

In [4]:
df = df.drop_duplicates(subset='text')
df.shape

(1674264, 27)

In [5]:
N = df.shape[0]
MaxUsed = 100

df = df.iloc[0:N]

In [6]:
#Length of review
df['reviewlength'] = df['text'].str.len()

In [7]:
#number of sentences
df['sentences'] = df["text"].str.count('\.')

In [8]:
#antal tal

# procentdel af store bogstaver

#procentdel af positive/negative meningsbærende ord --> Måske denne tages ud. Eller der laves liste med positive ord og en med negative


#gennemsnit længde af review
#one hot

In [8]:
#remove stop words
df['filtered_text'] = df.text.apply(remove_stopwords)

In [9]:
# Tokenize the text
df['tokenized_text'] = [simple_preprocess(line, deacc=True) for line in df['filtered_text']] 

In [10]:
#Stem the text
porter_stemmer = PorterStemmer()
df['stemmed_tokens'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokenized_text']]

In [11]:
df['stemmed_text'] = df['stemmed_tokens'].apply(' '.join)

In [12]:
#count of stars given
min_review = min(df['review_stars'].value_counts())

#dataframes for each star
df1 = df.loc[(df['review_stars'] == 1)]
df2 = df.loc[(df['review_stars'] == 2)]
df3 = df.loc[(df['review_stars'] == 3)]
df4 = df.loc[(df['review_stars'] == 4)]
df5 = df.loc[(df['review_stars'] == 5)]

#sample dataframe for each star
df1 = df1.sample(n=min_review,axis='rows')
df2 = df2.sample(n=min_review,axis='rows')
df3 = df3.sample(n=min_review,axis='rows')
df4 = df4.sample(n=min_review,axis='rows')
df5 = df5.sample(n=min_review,axis='rows')

In [13]:
#balanced reviews
df_word = pd.concat([df1, df2, df3, df4, df5])

In [14]:
Word = []
WordCounter = Counter(" ".join(df_word["stemmed_text"]).split()).most_common(MaxUsed)
for j in range(len(WordCounter)):
    Word.insert(j,WordCounter[j][0])

In [ ]:
Words = np.zeros((N,MaxUsed))

#one-hot encoding over the 100 most used words
for index, row in df.iterrows():
    for i in range(np.size(Words, axis=1)):
        if Word[i] in df['stemmed_text'].iloc[index]:
            Words[index,i] = 1

In [ ]:
#drop attributes with text
df = df.drop(['text','filtered_text','tokenized_text', 'stemmed_tokens',
       'stemmed_text'], axis=1)

In [ ]:
#dataframe for encoding
df_words = pd.DataFrame(Words)

#Encoding column names are changed
Names = list(range(0, MaxUsed))
ColNames = [str(n) for n in Names]

df_words.columns = ColNames
df_words

In [ ]:
#The encoding is added to the dataset
df = df.join(df_words)

In [ ]:
#label encoding for userID and businessID
df['user_id'] = df['user_id'].astype('category')
df['business_id'] = df['business_id'].astype('category')

df['UserCat'] = df['user_id'].cat.codes
df['BusinessCat'] = df['business_id'].cat.codes

In [ ]:
#encode date
df['date']= pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['DayOfWeek'] = df['date'].dt.dayofweek

In [ ]:
df_feature = df[['stars', 'review_count','review_stars','reviewlength', 'sentences', 'UserCat','BusinessCat', 'year', 'month', 'day', 'hour', 'minute', 'DayOfWeek', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']]

In [ ]:
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.2),max_features=1)
model.fit(df_feature)

In [ ]:
df['scores']=model.decision_function(df_feature)
df['anomaly']=model.predict(df_feature)

In [ ]:
df = df.drop(['reviewlength', 'sentences', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', 'UserCat', 'BusinessCat', 'year', 'month', 'day', 'hour', 'minute', 'DayOfWeek'], axis = 1)

In [26]:
#df[df['scores']>=0]

In [ ]:
path = 'C:\\Users\\Miche\\OneDrive - Danmarks Tekniske Universitet\\MMC\\2. Semester\\Social Data\\websites\\data'
os.chdir(path)
os.getcwd()

In [ ]:
csv_name = "AnomalyDetection.csv"
df.to_csv(csv_name, index=False)